In [1]:
!nvidia-smi

Mon May 24 10:17:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64       Driver Version: 440.64       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN RTX           Off  | 00000000:03:00.0 Off |                  N/A |
| 45%   64C    P2    83W / 280W |   8400MiB / 24219MiB |     25%      Default |
+-------------------------------+----------------------+----------------------+
|   1  TITAN RTX           Off  | 00000000:04:00.0 Off |                  N/A |
| 41%   49C    P2    62W / 280W |  10880MiB / 24220MiB |     23%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

### Import libraries

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.optim.lr_scheduler import ReduceLROnPlateau

import torchvision
from torchvision import transforms
import torchvision.models as models

import nibabel
import nibabel as nib
from nibabel.testing import data_path

from tqdm import tqdm_notebook
from tqdm.notebook import tqdm

from sklearn.metrics import roc_auc_score, f1_score

### Make custom dataset

In [4]:
class custom_dataset(Dataset):
    def __init__(self, type_='train'):
        # load all nii handle in a list
        self.labels = []
        self.images_list = []
        
        df = pd.read_csv(type_ + '.csv')
        files = df['imagename']

        
        labels_dict = {'pneumonia':0, 'COVID-19':1, 'normal':2}

        transform = transforms.Compose([
                              transforms.ToPILImage(),
                              transforms.Grayscale(num_output_channels=1),
                              transforms.Resize((600,600)),
                              transforms.RandomAffine(degrees=5,translate=(0.1,0.1)),
                              transforms.CenterCrop((512, 512)),
                              transforms.RandomHorizontalFlip(p=0.5),
                              transforms.ColorJitter(brightness=(0.9, 1.1)),
                              transforms.ToTensor(),
                              transforms.Normalize((0.5,),(0.5,))
])

        for i, file in tqdm(enumerate(files)):
            if_exist = os.path.exists(type_ + '/' + file)
            if if_exist:
                image = Image.open(type_ + '/' + file)
                image = np.array(image)
            
                # if len(image.shape)==2:
                image = torch.tensor(image)
                if len(image.shape)>2:
                  if image.shape[0] != 3 and image.shape[0] != 1:
                    image = image.permute(2, 0, 1)
                else:
                    image.unsqueeze_(0)
                self.images_list.append(transform(image))
                self.labels.append(labels_dict[df['class'][i]])

    def __len__(self):
        return len(self.images_list)

    def __getitem__(self, idx):
        # idx_ = np.random.choice(3)
        image = self.images_list[idx]
        target = self.labels[idx]
        
        return image, target

In [5]:
%%time
train_dataset = custom_dataset(type_='train')


CPU times: user 41min 49s, sys: 26.7 s, total: 42min 16s
Wall time: 5min 3s


In [6]:
%%time
val_dataset = custom_dataset(type_='val')


CPU times: user 8min 26s, sys: 7.53 s, total: 8min 33s
Wall time: 1min 5s


In [7]:
%%time
test_dataset = custom_dataset(type_='test')


CPU times: user 8min 36s, sys: 8.35 s, total: 8min 44s
Wall time: 1min 3s


In [8]:
batch_size = 8

In [9]:
test_dataloader = DataLoader(test_dataset, 
                             batch_size=batch_size,
                             shuffle=True, 
                             num_workers=0)

train_dataloader = DataLoader(train_dataset, 
                              batch_size=batch_size,
                              shuffle=True, 
                              num_workers=0)

val_dataloader = DataLoader(val_dataset, 
                            batch_size=batch_size,
                            shuffle=True, 
                            num_workers=0)

### Check distribution per class

In [10]:
labels = []
for i, batch in tqdm_notebook(enumerate(train_dataloader)):
    labels = np.append(labels, batch[1])
    
print('Train dataset')
print('Total size:', len(labels))
print('First class: ', len(labels[labels==0]))
print('Second class: ', len(labels[labels==1]))
print('Third class: ', len(labels[labels==2]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



Train dataset
Total size: 10396
First class:  4296
Second class:  363
Third class:  5737


In [11]:
labels = []
for i, batch in tqdm_notebook(enumerate(val_dataloader)):
    labels = np.append(labels, batch[1])
    
print('Val dataset')
print('Total size:', len(labels))
print('First class: ', len(labels[labels==0]))
print('Second class: ', len(labels[labels==1]))
print('Third class: ', len(labels[labels==2]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



Val dataset
Total size: 2239
First class:  918
Second class:  89
Third class:  1232


In [12]:
labels = []
for i, batch in tqdm_notebook(enumerate(test_dataloader)):
    labels = np.append(labels, batch[1])

print('Test dataset')
print('Total size:', len(labels))
print('First class: ', len(labels[labels==0]))
print('Second class: ', len(labels[labels==1]))
print('Third class: ', len(labels[labels==2]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



Test dataset
Total size: 2222
First class:  917
Second class:  61
Third class:  1244


In [13]:
device = "cuda:0"

In [14]:
model = models.densenet161(pretrained=True)
model.features.conv0 = nn.Conv2d(1, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.classifier = nn.Linear(in_features=2208, out_features=3, bias=True)

In [15]:
model = model.to(device)

In [16]:
print('Number of batches in train loader: ', len(train_dataloader))

Number of batches in train loader:  1300


In [17]:
def run_epoch(model, dataloader, optimizer, criterion, lr_scheduler, phase='train'):
    is_train = (phase == 'train')
    if is_train:
        model.train()
    else:
        model.eval()
    
    epoch_loss = 0

    # variables for calculating accuracy
    n_predicted, n_true_predicted = 0, 0
    
    with torch.set_grad_enabled(is_train):
        
        answers_true = []
        answers_pred = []
        
        for i, batch in tqdm_notebook(enumerate(dataloader)):
            # unpack batch
            features, labels = batch
            features, labels = features.to(device), labels.to(device)
            
            # make prediction
            pred = model.forward(features)
            
            # reshape prediction to [-1, output_dim]
            pred = pred.view(-1, pred.size(-1))
            
            # reshape gt labels to [-1, ]
            gt = labels.reshape(-1)
            
            # calculate loss
            loss = criterion(pred, gt)
            
            if is_train:
                # make optimization step
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                
            # calculate accuracy
            n_true_predicted += (pred.argmax(-1) == gt).sum().item()  # exclude pad token
            n_predicted += len(gt)
            
            answers_true = np.append(answers_true, gt.cpu())
            answers_pred = np.append(answers_pred, pred.argmax(-1).cpu())   
            
            # log per-batch train metrics
                
            epoch_loss += loss.item()
            
        answers_true, answers_pred = np.int8(answers_true), np.int8(answers_pred)
        
        f1 = f1_score(answers_true, answers_pred, average='weighted')
        gt_one_hot = np.eye(np.max(answers_true)+1)[answers_true]
        pred_one_hot = np.eye(np.max(answers_pred)+1)[answers_pred]
        auc = roc_auc_score(gt_one_hot, pred_one_hot, average='macro')

        average_loss = epoch_loss / len(dataloader)
        average_accuracy = n_true_predicted / n_predicted
        
        if phase=='val' and (lr_scheduler is not None):
            lr_scheduler.step(average_loss)
        
        # log per-epoch metrics

        return average_loss, average_accuracy, auc, f1

In [18]:
!rm -rf logs1

In [19]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
path = "./logs1"
best_model_path = path + "/model.pth"

In [20]:
writer = SummaryWriter(log_dir=path+"/tensorboard")
scheduler = ReduceLROnPlateau(optimizer, patience=8, factor=0.2, verbose=True)

In [21]:
n_epochs = 40

best_val_loss = float('+inf')
for epoch in range(n_epochs):
    train_loss, train_accuracy, train_auc, train_f1 = run_epoch(model, train_dataloader, optimizer, criterion, scheduler, phase='train')
    val_loss, val_accuracy, val_auc, val_f1 = run_epoch(model, val_dataloader, None, criterion, scheduler, phase='val')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), best_model_path)

    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f} | Train accuracy: {train_accuracy * 100:.2f} | Train AUC: {train_auc * 100:.2f} | Train F1: {train_f1 * 100:.2f}')
    print(f'\t Val. Loss: {val_loss:.3f} |  Val. accuracy: {val_accuracy * 100:.2f} | Val AUC: {val_auc * 100:.2f} | Val F1: {val_f1 * 100:.2f}')

    writer.add_scalar('Loss/train', train_loss, epoch)
    writer.add_scalar('Loss/val', val_loss, epoch)
    writer.add_scalar('Accuracy/train', train_accuracy, epoch)
    writer.add_scalar('Accuracy/val', val_accuracy, epoch)   
    writer.add_scalar('AUC/train', train_auc, epoch)
    writer.add_scalar('AUC/val', val_auc, epoch)
    writer.add_scalar('F1/train', train_f1, epoch)
    writer.add_scalar('F1/val', val_f1, epoch)
    writer.flush()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



Epoch: 01
	Train Loss: 0.617 | Train accuracy: 75.27 | Train AUC: 67.69 | Train F1: 73.93
	 Val. Loss: 0.850 |  Val. accuracy: 65.34 | Val AUC: 58.62 | Val F1: 60.09


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



Epoch    39: reducing learning rate of group 0 to 4.0000e-05.
Epoch: 39
	Train Loss: 0.007 | Train accuracy: 99.86 | Train AUC: 99.73 | Train F1: 99.86
	 Val. Loss: 0.476 |  Val. accuracy: 88.88 | Val AUC: 90.15 | Val F1: 89.04


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



Epoch: 40
	Train Loss: 0.006 | Train accuracy: 99.83 | Train AUC: 99.76 | Train F1: 99.83
	 Val. Loss: 0.455 |  Val. accuracy: 90.89 | Val AUC: 89.86 | Val F1: 90.85


In [22]:
model = models.densenet161(pretrained=True)
model.features.conv0 = nn.Conv2d(1, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.classifier = nn.Linear(in_features=2208, out_features=3, bias=True)
model.load_state_dict(torch.load('logs1/model.pth'))

<All keys matched successfully>

In [23]:
device = 'cuda:0'
model = model.to(device)

In [24]:
test_loss, test_accuracy, test_auc, test_f1 = run_epoch(model, test_dataloader, None, criterion, scheduler, phase='val')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [25]:
 print(f'\t Test Loss: {test_loss:.3f} |  Test accuracy: {test_accuracy * 100:.2f} | Test AUC: {test_auc * 100:.2f} | Test F1: {test_f1 * 100:.2f}')

	 Test Loss: 0.263 |  Test accuracy: 91.27 | Test AUC: 88.79 | Test F1: 91.30


In [26]:
model = model.eval()

In [27]:
labels_true = []
labels_predict = []

for i, batch in tqdm_notebook(enumerate(test_dataloader)):
    features, labels = batch
    features = features.to(device)
    
    pred = model.forward(features)
    labels = labels.numpy()
    labels_true = np.append(labels_true, labels)
    
    idx = np.where(labels==1)[0]
    
    pred = pred.view(-1, pred.size(-1))
    
    y_pred = pred.argmax(-1)
    labels_predict = np.append(labels_predict, y_pred.cpu().numpy())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


RuntimeError: CUDA out of memory. Tried to allocate 18.00 MiB (GPU 0; 23.65 GiB total capacity; 19.43 GiB already allocated; 9.06 MiB free; 19.53 GiB reserved in total by PyTorch)

In [ ]:
idx = np.where(labels_true==1)[0]

In [ ]:
idx

In [ ]:
len(idx)

In [ ]:
labels_true[idx]

In [ ]:
labels_predict[idx]

In [ ]:
acc = np.sum(labels_true[idx]==labels_predict[idx])

In [ ]:
acc = acc / len(idx)

In [ ]:
acc